In [1]:
import json
import pandas as pd
from tqdm import tqdm

import seaborn as sns
import matplotlib.pyplot as plt

import urllib.request
from bs4 import BeautifulSoup

from pygments import highlight
from pygments.lexers import JsonLexer
from pygments.formatters import TerminalFormatter
from google_play_scraper import Sort, reviews, app, reviews_all

In [2]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

In [3]:
fname = r'C:\Users\srinivasan.p.1\Anaconda3\html\opioid.html'
HtmlFile = open(fname, 'r', encoding='utf-8')
source_code = HtmlFile.read()

soup = BeautifulSoup(source_code, "html.parser")
soup.prettify()

'<!DOCTYPE html>\n<html lang="en" xmlns="http://www.w3.org/1999/xhtml">\n <head>\n  <meta charset="utf-8"/>\n  <title>\n  </title>\n </head>\n <body>\n  <div class="ZmHEEd" jsaction="rcuQ6b:npT2md;M8vzZb:.CLIENT" jscontroller="i2u2Pb" jsname="O2DNWb">\n   <div class="ImZGtf mpg5gc">\n    <c-wiz autoupdate="" data-node-index="1;0" data-p="%.@.true]" jsaction="aM6rcc:MRWqkd" jscontroller="JpEzfb" jsdata="deferred-c139" jsmodel="hc6Ubd" jsrenderer="PAQZbb" jsshadow="">\n     <div class="uMConb V2Vq5e POHYmb-eyJpod YEDFMc-eyJpod y1APZe-eyJpod drrice" jslog="38003; 1:500|CAIaUQooEiYKIGF1LmVkdS5hbnpjYS5vcGlvaWRjYWxjdWxhdG9yYXBwEAEYAxAAMhMIxo7r6JOa7QIVDG5mCh3dwgZAigENCAASCQoFZW4tVVMQAKoCVRpTCAASKAomCiBhdS5lZHUuYW56Y2Eub3Bpb2lkY2FsY3VsYXRvcmFwcBABGANKEwjGjuvok5rtAhUMbmYKHd3CBkD6AQ8KDQgAEgkKBWVuLVVTEAA=; track:click,impression">\n      <div class="Vpfmgd">\n       <div class="uzcko">\n        <div class="N9c7d eJxoSc">\n         <span class="yNWQ8e K3IMke buPxGf" style="height: 160px; width: 16

In [4]:
##get all app div tags
app_divs = []
app_packages = []

divs = soup.find_all('div',{'class':'b8cIId ReQCgd Q9MA7b'})

##Loop through each div to find the app ID
for eachdiv in divs:
    ##Retrieve app ID
    app_ID = eachdiv.find('a').get('href')
    app_packages.append(app_ID.split("/store/apps/details?id=", 1)[-1])

In [10]:
app_reviews=[]
ratings = [0,1,2,3,4,5]

for rating in ratings:
    for ap in tqdm(app_packages):
        rvs = reviews_all(
            ap,
            sort=Sort.MOST_RELEVANT,
            filter_score_with=rating
        )
        for r in rvs:
            r['appId'] = ap
        app_reviews.extend(rvs)

100%|██████████| 247/247 [09:55<00:00,  2.41s/it]


In [11]:
def print_json(json_object):
    json_str = json.dumps(
        json_object,
        indent=2,
        sort_keys=True,
        default=str
    )
    print (highlight(json_str, JsonLexer(), TerminalFormatter()))

In [12]:
app_reviews_df = pd.DataFrame(app_reviews)
col_list = ['appId','userName','at','content','score','thumbsUpCount']
app_reviews_df[col_list].to_csv('reviewsall_opioid.csv', index=None, header=True)